In [1]:
import sys
import os
searchPath=os.path.abspath('..')
sys.path.append(searchPath)
from math import exp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
# Loading the dataset in python with the name df and displaying first 20 rows: 
df=pd.read_csv("e-shop clothing 2008.csv",sep = ';')
df.head(20)

,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,price 2,page
0,2008,4,1,1,29,1,1,A13,1,5,1,28,2,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,2,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,2,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1
5,2008,4,1,6,29,1,3,C56,6,1,2,57,1,4
6,2008,4,1,7,29,1,3,C57,5,1,2,33,2,4
7,2008,4,1,8,29,1,4,P67,9,5,1,38,1,4
8,2008,4,1,9,29,1,4,P82,6,4,2,48,1,5
9,2008,4,1,1,29,2,2,B31,9,5,1,57,1,2


In [3]:
df=df[0:1000]

In [4]:
test=df['price 2']

In [5]:
train=df[df.columns.drop('price 2')]
train=train[train.columns.drop('page 2 (clothing model)')]
train=train[train.columns.drop('month')]
train=train[train.columns.drop('day')]
train=train[train.columns.drop('year')]

In [6]:
train.shape

(1000, 9)

In [7]:
train=train.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [8]:
train=pd.concat([train,test],axis=1)

In [9]:
# data
def create_data():
    data = np.array(train.iloc[:1000,:])
    return data[:,[0,1,2,3,4,5,6,7,8]], data[:,-1]

In [10]:
from sklearn.linear_model import LogisticRegression
from math import exp
class LogisticRegressionClassifier(object):
    def __init__(self, max_iter=200, learning_rate=0.01):
        # 最大迭代次数
        self.max_iter = max_iter
        # 学习率
        self.learning_rate = learning_rate

    # sigmoid函数
    def sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # 处理训练数据，增加一列，为了weight和bias合并处理
    def data_matrix(self, X):
        data_mat = []
        for d in X:
            data_mat.append([1.0, *d])
        return data_mat


    def fit(self, X, y):
        data_mat = self.data_matrix(X)
        # self.weights包含了weight和bias合并处理
        self.weights = np.zeros((len(data_mat[0]), 1), dtype=np.float32)

        for iter_ in range(self.max_iter):
            for i in range(len(X)):
                result = self.sigmoid(np.dot(data_mat[i], self.weights))
                error = y[i] - result
        
                self.weights += self.learning_rate * error * np.transpose([data_mat[i]])
        print('LogisticRegression Model(learning_rate={},max_iter={})'.format(self.learning_rate, self.max_iter))

    # 计算准确度
    def score(self, X_test, y_test):
        right = 0
        X_test = self.data_matrix(X_test)
        for x, y in zip(X_test, y_test):
            result = np.dot(x, self.weights)
            if (result > 0 and y == 1) or (result < 0 and y == 0):
                right += 1
        return right / len(X_test)

In [11]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
lr_clf = LogisticRegressionClassifier()
lr_clf.fit(X_train, y_train)

LogisticRegression Model(learning_rate=0.01,max_iter=200)


In [13]:
lr_clf.score(X_test, y_test)

0.545